# Instore Prices

This is a followup analysis to simplify v1. It is a complement, not a correction

## Config

In [34]:
import matplotlib.pyplot as  plt
# import numpy as np
import pandas as pd
import seaborn as sns
import sys
from scipy.stats import ttest_ind

sys.path.append('c:\\Users\\Jordi Cremades\\Documents\\Repos\\central-node')

# from utils import dataset_meta_stats
# from utils import dataset_stats
from utils import query_engines

# dms = dataset_meta_stats.DatasetMetaStats() 
# ds = dataset_stats.DatasetStats()

## [X] Identify top Groceries cities

In [14]:
START_DATE = "'2023-12-01'"
END_DATE = "'2024-06-01'"
TOP = 100

params = [
    # dates
    {'name':'start_date', 'value': str(START_DATE)},
    {'name':'end_date', 'value': str(END_DATE)},
    {'name':'top', 'value': str(TOP)}
    
]

q = query_engines.QueryEngines(
    query='top_cities_v2.sql',
    params=params,
    printq=None,
    output_file='top_cities_v2',
    load_from_output_file=None
    
)

df = q.query_run_starbust()
df

c:\Users\Jordi Cremades\Documents\Repos\central-node\utils\query_engines.py:162: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


Open the following URL in browser for the external authentication:
https://starburst.g8s-data-platform-prod.glovoint.com/oauth2/token/initiate/07e96399ff3d1eb41969d3609dca0e88204e2917ae6ffc2855c6a0c66959ae86


,order_city_code,order_country_code,n_orders
0,MAD,ES,866839
1,BUC,RO,834005
2,TBI,GE,709015
3,CAS,MA,687670
4,WAW,PL,606077
...,...,...,...
95,MBS,KE,27803
96,CST,ES,27369
97,FES,MA,27215
98,QYY,PL,26891


## [X] Compute raw metrics

We need to cut the raw_metric_v2 query, into two separate queries and later we will join. The first part are the metrics related to orders.

In [5]:
START_DATE = "'2023-12-01'"
END_DATE = "'2024-06-01'"
TOP = 100

params = [
    # dates
    {'name':'start_date', 'value': str(START_DATE)},
    {'name':'end_date', 'value': str(END_DATE)},
    {'name':'top', 'value': str(TOP)}
    
]

q = query_engines.QueryEngines(
    query='raw_metrics_v2_part1.sql',
    params=params,
    printq=None,
    output_file='raw_metrics_v2_part1',
    load_from_output_file='raw_metrics_v2_part1'
    
)

df1 = q.query_run_starbust()
df1

,country,city,p_creation_date,nswg_variant,ipg_variant,groceries_orders_count,groceries_ret_orders_count,stores_ret_orders_count,groceries_gmv_sum
0,RO,IAS,2024-05-15,Variant 1,InStorePrices,95,81,56,2677.945579
1,RO,IAS,2023-12-31,Variant 1,NaN,435,289,200,8533.579776
2,RO,IAS,2024-03-18,Variant 1,Control Group,401,313,249,8926.560891
3,RO,IAS,2024-03-04,Control Group,NaN,52,43,34,1443.806922
4,RO,IAS,2024-05-01,Control Group,NaN,76,59,45,2139.752792
...,...,...,...,...,...,...,...,...,...
79649,PL,QLU,2024-05-27,Variant 1,NaN,13,6,6,143.336205
79650,PL,QLU,2023-12-25,Variant 1,NaN,6,3,2,83.460005
79651,MA,BSR,2024-05-26,NaN,NaN,7,4,2,112.124988
79652,RO,BUC,2024-02-05,Variant 1,InStorePrices,1,1,1,10.109699


The second part is related to sessions metrics

In [71]:
START_DATE = "'2023-03-01'"
END_DATE = "'2024-06-01'"
TOP = 100

params = [
    # dates
    {'name':'start_date', 'value': str(START_DATE)},
    {'name':'end_date', 'value': str(END_DATE)},
    {'name':'top', 'value': str(TOP)}
    
]

q = query_engines.QueryEngines(
    query='raw_metrics_v2_part2.sql',
    params=params,
    printq=None,
    output_file='raw_metrics_v2_part2',
    load_from_output_file=None
    
)

df2 = q.query_run_starbust()
df2

c:\Users\Jordi Cremades\Documents\Repos\central-node\utils\query_engines.py:162: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


Open the following URL in browser for the external authentication:
https://starburst.g8s-data-platform-prod.glovoint.com/oauth2/token/initiate/63ad02c2d4d18760fa40e32e751c7b2d5cc5e4019da9cba176f8215346659747


,country,city,p_creation_date,nswg_variant,ipg_variant,groceries_category_opened_session_count,groceries_category_opened_event_count,customer_id
0,MA,RBT,2024-03-04,None,None,87,87,81
1,RO,BUC,2023-12-03,None,None,7176,7176,6379
2,UA,CHK,2024-04-26,Variant 1,Control Group,233,233,216
3,PL,WAW,2023-12-31,Variant 1,None,6031,6031,5151
4,NG,LOS,2023-12-09,None,None,1020,1020,845
...,...,...,...,...,...,...,...,...
80921,IT,PMO,2024-05-28,None,None,5,5,5
80922,MA,DRB,2024-05-13,None,None,7,7,6
80923,ES,MAL,2024-05-22,Control Group,InStorePrices,1,1,1
80924,GE,TBI,2024-05-22,None,Control Group,1,1,1


In [27]:
df_merged = pd.merge(df1,df2, on=['country','city','p_creation_date','nswg_variant','ipg_variant'])

#fillna
df_merged['nswg_variant'].fillna('not_found', inplace=True)
df_merged['ipg_variant'].fillna('not_found', inplace=True)

#metrics
df_merged['perc_cvr'] = df_merged['groceries_orders_count']/df_merged['groceries_sw_session_count']
df_merged['perc_ret'] = df_merged['groceries_ret_orders_count']/df_merged['groceries_orders_count']
df_merged['aov'] = df_merged['groceries_gmv_sum']/df_merged['groceries_orders_count']
df_merged['groceries_session_per_customer']  = df_merged['groceries_gmv_sum']/df_merged['groceries_orders_count'] XXXXXXXXXXXX

df_final = df_merged[[
    'country',
    'city',
    'p_creation_date',
    'nswg_variant',
    'ipg_variant',
    'groceries_orders_count',
    'groceries_sw_session_count',
    'groceries_gmv_sum',
    'groceries_ret_orders_count',
    'groceries_session_per_customer',
    'perc_cvr',
    'perc_ret',
    'aov']]

cond = df_final['nswg_variant'] == 'Variant 1'
df_final[cond].to_csv('outputs/raw_results.csv')

## [X] Compute P-Values and aggregates

In [51]:
tmp_df = df_final[cond].copy()

In [61]:
metrics = [
    'groceries_orders_count', 
    'groceries_sw_session_count', 
    'groceries_gmv_sum', 
    'groceries_ret_orders_count',
    'groceries_session_per_customer',
    'perc_cvr', 
    'perc_ret', 
    'aov'
]


aggregated = tmp_df.groupby(['country', 'city', 'ipg_variant']).agg({
    'groceries_orders_count': 'mean',
    'groceries_sw_session_count': 'mean',
    'groceries_gmv_sum': 'mean',
    'groceries_ret_orders_count': 'mean',
    'groceries_session_per_customer':'mean'
    'perc_cvr': 'mean',
    'perc_ret': 'mean',
    'aov': 'mean'
}).reset_index()

control_data = aggregated[aggregated['ipg_variant'] == 'Control Group']
variant_data = aggregated[aggregated['ipg_variant'] == 'InStorePrices']

merged_data = pd.merge(
    control_data, variant_data, 
    on=['country', 'city'], 
    suffixes=('_control', '_variant')
)

merged_data


,country,city,ipg_variant_control,groceries_orders_count_control,groceries_sw_session_count_control,groceries_gmv_sum_control,groceries_ret_orders_count_control,perc_cvr_control,perc_ret_control,aov_control,ipg_variant_variant,groceries_orders_count_variant,groceries_sw_session_count_variant,groceries_gmv_sum_variant,groceries_ret_orders_count_variant,perc_cvr_variant,perc_ret_variant,aov_variant
0,AM,YRV,Control Group,175.931624,147.581197,3315.156706,135.393162,1.567259,0.775204,18.596402,InStorePrices,38.840000,34.584000,717.823923,30.168000,1.428949,0.790730,18.730025
1,BG,SOF,Control Group,459.534483,437.913793,11410.923329,344.224138,1.092563,0.754057,25.103530,InStorePrices,100.246154,96.900000,2573.200539,74.646154,1.046175,0.752242,25.369814
2,CI,ABJ,Control Group,357.327586,428.275862,4850.754643,292.948276,0.860156,0.820762,13.530551,InStorePrices,93.529915,108.324786,1341.976579,78.059829,0.898621,0.838478,14.335697
3,CI,ABN,Control Group,846.179487,811.042735,10822.182129,697.102564,1.165775,0.825538,12.754984,InStorePrices,210.707317,192.601626,2727.659512,173.869919,1.212630,0.827975,12.729736
4,ES,ALC,Control Group,184.504348,228.313043,4654.720000,141.426087,0.945979,0.758693,24.592439,InStorePrices,48.678261,57.721739,1202.220696,37.634783,1.001040,0.766055,24.125593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,UA,POL,Control Group,83.165217,108.321739,785.410843,55.069565,0.830915,0.664005,9.438410,InStorePrices,22.026087,27.365217,222.573938,15.121739,0.870235,0.690735,10.094704
96,UA,VNT,Control Group,225.147826,198.043478,2521.570892,169.504348,1.291558,0.756559,11.206422,InStorePrices,55.033898,48.822034,607.783399,41.491525,1.283005,0.754406,10.812476
97,UA,ZHY,Control Group,99.530435,118.347826,1094.598979,70.965217,0.935306,0.714825,10.999397,InStorePrices,24.000000,28.417391,292.844088,16.913043,0.980718,0.709506,12.221425
98,UA,ZPR,Control Group,168.878261,197.321739,1959.063072,130.086957,0.959808,0.773622,11.600389,InStorePrices,39.669565,49.026087,451.309732,29.939130,0.912429,0.756521,11.377531


In [58]:
results = []

for index, row in merged_data.iterrows():
    country = row['country']
    city = row['city']
    result_row = {'country': country, 'city': city}
    
    for metric in metrics:
        control_value = row[f'{metric}_control']
        variant_value = row[f'{metric}_variant']
        
        # Calculate increment
        increment = (variant_value - control_value) / control_value * 100 if control_value != 0 else None
        
        # Perform t-test
        control_sample = tmp_df[(tmp_df['country'] == country) & (tmp_df['city'] == city) & (tmp_df['ipg_variant'] == 'Control Group')][metric]
        variant_sample = tmp_df[(tmp_df['country'] == country) & (tmp_df['city'] == city) & (tmp_df['ipg_variant'] == 'InStorePrices')][metric]
        
        t_stat, p_value = ttest_ind(control_sample, variant_sample, equal_var=False)
        
        # Append to results
        result_row.update({
            f'{metric}_control': control_value,
            f'{metric}_variant': variant_value,
            f'increment_{metric}': increment,
            f'p_value_{metric}': p_value
        })
    
    results.append(result_row)


In [62]:
final_df = pd.DataFrame(results)
final_df.to_csv('outputs/t_test.csv')